$$ {Volatility Prediction} $$

Call Option : price = $B \cdot (F\cdot N(d_1) - K\cdot N(d_2))$

Put Option : $ B \cdot (K \cdot N(-d_2) - F \cdot N(-d_1)) $

In [ ]:
import numpy as np
import scipy.stats as si
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.svm import SVR
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.metrics import mean_squared_error, explained_variance_score
from sklearn.utils import shuffle
from sklearn.preprocessing import StandardScaler
def N(x): # cdf for standard normal distribution 
  return si.norm.cdf(x, 0., 1.)
 
def european_option(S, K, T, r, d, sigma, call_option = 1):
  # S is spot price
  # K is strike price
  # T is time to maturity (unit : years)
  # r is risk-free rate per year
  # d is dividend rate per year 
  # sigma is volatility per year
  # call_option = 1 if call, 0 if put option
 
  F = S*np.exp((r-d)*T) # forward price
  B = np.exp(-r*T) # discount factor
  d1 = (np.log(F/K) + 0.5*sigma**2*T)/(sigma*T**.5)
  d2 = (np.log(F/K) - 0.5*sigma**2*T)/(sigma*T**.5)
  if call_option == 1:
    result = B*(F*N(d1) - K*N(d2))
  if call_option == 0:
    result = B*(K*N(-d2) - F*N(-d1))
  return B*result

In [ ]:
# Simulate Option data
np.random.seed(42)
draws = 10**3                                 # number of simulation
S = np.random.rand(draws)*100                # is spot price
K = np.random.randint(50, 150, draws)*.01*S   # is strike price
T = np.random.randint(10, 300, draws)*.01     #is time to maturity (unit : years)
r = np.random.rand(draws)*.1 # is risk-free rate per year
d = np.random.rand(draws)*.1 # is dividend rate per year
sigma = np.random.rand(draws) # is volatility per year
opt_type = np.random.choice([0,1], draws)

In [ ]:
# simulate option prices
opt_price = []
for i in range(draws):
  p = european_option(S[i], K[i], T[i], r[i], d[i], sigma[i], opt_type[i])
  opt_price.append(p)
  if (i % 5000 == 0):
    print('Gnerated {} options'.format(i))
 
# create dataframe
options = pd.DataFrame({'S':S,
                        'K':K,
                        'T':T,
                        'r':r,
                        'd':d,
                        'sigma':sigma,
                        'type':opt_type,
                        'price':opt_price})
 
print(options.head())
print(options.shape)
options

Gnerated 0 options
           S          K     T         r         d     sigma  type      price
0  37.454012  35.955851  2.96  0.087205  0.016812  0.494521     0   5.354697
1  95.071431  57.993573  2.17  0.052723  0.047495  0.121695     0   0.007537
2  73.199394  81.251328  1.15  0.063254  0.005522  0.588094     1  15.783524
3  59.865848  77.226944  1.67  0.064447  0.094238  0.316078     0  19.196255
4  15.601864  21.374554  1.30  0.081980  0.007482  0.626968     0   6.233007
(1000, 8)


S           K     T         r         d     sigma  type      price
0    37.454012   35.955851  2.96  0.087205  0.016812  0.494521     0   5.354697
1    95.071431   57.993573  2.17  0.052723  0.047495  0.121695     0   0.007537
2    73.199394   81.251328  1.15  0.063254  0.005522  0.588094     1  15.783524
3    59.865848   77.226944  1.67  0.064447  0.094238  0.316078     0  19.196255
4    15.601864   21.374554  1.30  0.081980  0.007482  0.626968     0   6.233007
..         ...         ...   ...       ...       ...       ...   ...        ...
995   9.158207    9.799282  2.17  0.089189  0.036165  0.621106     1   2.568165
996  91.731358   77.971654  1.23  0.030605  0.012538  0.301800     1  19.476347
997  13.681863   16.144598  0.31  0.055188  0.025361  0.899743     1   1.868905
998  95.023735  119.729907  2.23  0.078941  0.064209  0.628407     1  20.665880
999  44.600577   63.332820  1.80  0.041622  0.027554  0.438649     1   4.851312

[1000 rows x 8 columns]

In [ ]:
X,y = options[options.columns.difference(['sigma'])], options['sigma']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)
scalerX = StandardScaler()
scalerX.fit(X_train)
X_train_scaled = scalerX.transform(X_train)
X_test_scaled = scalerX.transform(X_test)
scalerY = StandardScaler()
scalerY.fit(y_train)
y_train_scaled = scalerY.transform(y_train)
y_test_scaled = scalerY.transform(y_test)

reg = SVR()
reg.fit(X_train_scaled, y_train_scaled)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
y_train_hat_scaled = reg.predict(X_train_scaled)
y_train_hat = scalerY.inverse_transform(y_train_hat_scaled)
print('R_square:', r2_score(y_train, y_train_hat))
y_test_hat_scaled = reg.predict(X_test_scaled)
y_test_hat = scalerY.inverse_transform(y_test_hat_scaled)
print('R_square:', r2_score(y_test, y_test_hat))

R_square: 0.5973338536843651
R_square: 0.49837445467969166


In [ ]:
 C_ep_ga_data = pd.DataFrame(columns = ('C', 'epsilon', 'gamma', 'training r2', 'test r2'))

training_r2score = []
test_r2score = []
C_settings = [1, 100]
epsilon_settings = [0.001, 0.01, 0.1]
gamma_settings = [0.01, 0.1]

for C in C_settings:
  for epsilon in epsilon_settings:
    for gamma in gamma_settings:
      # build the model
      reg = SVR(C=C, kernel = 'rbf', epsilon=epsilon, gamma=gamma)
      reg.fit(X_train_scaled, y_train_scaled)

      # r2 on the training set
      y_train_hat = scalerY.inverse_transform(reg.predict(X_train_scaled))
      training_r2score.append(r2_score(y_train, y_train_hat))

      # r2 on the test set
      y_test_hat = scalerY.inverse_transform(reg.predict(X_test_scaled))
      test_r2score.append(r2_score(y_test, y_test_hat))

      i = [C, epsilon, gamma, r2_score(y_train, y_train_hat), r2_score(y_test, y_test_hat)]

      C_ep_ga_data.loc[len(C_ep_ga_data)] = i
C_ep_ga_data

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C  epsilon  gamma  training r2   test r2
0     1.0    0.001   0.01     0.204310  0.194981
1     1.0    0.001   0.10     0.540725  0.475233
2     1.0    0.010   0.01     0.204612  0.195448
3     1.0    0.010   0.10     0.540839  0.476346
4     1.0    0.100   0.01     0.205281  0.198945
5     1.0    0.100   0.10     0.543887  0.466384
6   100.0    0.001   0.01     0.597981  0.567637
7   100.0    0.001   0.10     0.859926  0.693819
8   100.0    0.010   0.01     0.598332  0.569101
9   100.0    0.010   0.10     0.860687  0.697323
10  100.0    0.100   0.01     0.597275  0.570019
11  100.0    0.100   0.10     0.865319  0.707075

In [ ]:
C_ep_ga_data = pd.DataFrame(columns = ('C', 'epsilon', 'gamma', 'training rmse', 'test rmse'))
  
training_rmsescore = []
test_rmsescore = []
C_settings = [1, 100]
epsilon_settings = [0.001, 0.01, 0.1]
gamma_settings = [0.01, 0.1]

for C in C_settings:
  for epsilon in epsilon_settings:
    for gamma in gamma_settings:
      # build the model
      reg = SVR(C=C, kernel = 'rbf', epsilon=epsilon, gamma=gamma)
      reg.fit(X_train_scaled, y_train_scaled)

      # rmse on the training set
      y_train_hat = scalerY.inverse_transform(reg.predict(X_train_scaled))
      training_rmsescore.append(mean_squared_error(y_train, y_train_hat))

      # rmse on the test set
      y_test_hat = scalerY.inverse_transform(reg.predict(X_test_scaled))
      test_rmsescore.append(mean_squared_error(y_test, y_test_hat))

      i = [C, epsilon, gamma, mean_squared_error(y_train, y_train_hat), mean_squared_error(y_test, y_test_hat)]

      C_ep_ga_data.loc[len(C_ep_ga_data)] = i
C_ep_ga_data

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C  epsilon  gamma  training rmse  test rmse
0     1.0    0.001   0.01       0.066317   0.062497
1     1.0    0.001   0.10       0.038278   0.040740
2     1.0    0.010   0.01       0.066291   0.062461
3     1.0    0.010   0.10       0.038269   0.040654
4     1.0    0.100   0.01       0.066236   0.062189
5     1.0    0.100   0.10       0.038015   0.041427
6   100.0    0.001   0.01       0.033506   0.033566
7   100.0    0.001   0.10       0.011674   0.023770
8   100.0    0.010   0.01       0.033477   0.033453
9   100.0    0.010   0.10       0.011611   0.023498
10  100.0    0.100   0.01       0.033565   0.033381
11  100.0    0.100   0.10       0.011225   0.022741

In [ ]:
num_training = int(0.8 * len(X))
X_train, y_train = X[:num_training], y[:num_training]
X_test, y_test = X[:num_training], y[:num_training]

sv_regressor = SVR(kernel='rbf', C=100.0, epsilon=0.1, gamma=0.10)

sv_regressor.fit(X_train, y_train)

y_pred = sv_regressor.predict(X_test)
X_test

K          S     T         d      price         r  type
0     35.955851  37.454012  2.96  0.016812   5.354697  0.087205     0
1     57.993573  95.071431  2.17  0.047495   0.007537  0.052723     0
2     81.251328  73.199394  1.15  0.005522  15.783524  0.063254     1
3     77.226944  59.865848  1.67  0.094238  19.196255  0.064447     0
4     21.374554  15.601864  1.30  0.007482   6.233007  0.081980     0
..          ...        ...   ...       ...        ...       ...   ...
795   65.409293  87.212391  0.99  0.032568  22.291586  0.083137     1
796  112.786312  93.211828  1.12  0.039414  26.570557  0.084233     0
797   54.817919  56.513318  2.59  0.060333  22.793963  0.037324     1
798   80.114845  69.665082  0.17  0.080168   0.347006  0.063199     1
799  109.777426  92.249938  1.62  0.046952  35.275804  0.009537     0

[800 rows x 7 columns]

In [ ]:
mse = mean_squared_error(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)
test_data = [38.852172, 37.454012, 0.44, 0.061208, 9.180301, 0.040804, 0]
print('Volatility: %.2f%%' % (sv_regressor.predict([test_data])[0]*100))
print('MSE:',mse)
print('EVS:',evs)

Volatility: 53.48%
MSE: 0.008205501656895743
EVS: 0.8985428911410713


In [ ]:
mse = mean_squared_error(y_test, y_pred)

evs = explained_variance_score(y_test, y_pred)
#0.494521 0.121695
test_data = [35.955851, 37.454012,	2.96,	 0.016812,	5.354697,	0.087205,	0]
print('Volatility: %.2f%%' % (sv_regressor.predict([test_data])[0]*100))
print('MSE:',mse)
print('EVS:',evs)

Volatility: 39.41%
MSE: 0.008205501656895743
EVS: 0.8985428911410713


In [ ]:
mse = mean_squared_error(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)
test_data = [57.9, 95.1, 2.17, 0.047495, 0.007537, 0.052723, 0]
print('Volatility: %.2f%%' % (sv_regressor.predict([test_data])[0]*100))
print('MSE:',mse)
print('EVS:',evs)

Volatility: 22.30%
MSE: 0.008205501656895743
EVS: 0.8985428911410713


In [ ]:
mse = mean_squared_error(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)
#0.494521 0.121695
test_data = [81.251328, 73.199394,	1.15,	0.005522,	15.783524,	0.063254,	1]
print('Volatility: %.2f%%' % (sv_regressor.predict([test_data])[0]*100))
print('MSE:',mse)
print('EVS:',evs)

Volatility: 54.26%
MSE: 0.008205501656895743
EVS: 0.8985428911410713


In [ ]:
mse = mean_squared_error(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)
test_data = [21.374554,	15.601864, 1.30,	0.007482,	6.233007, 0.0081980 ,0]
print('Volatility: %.2f%%' % (sv_regressor.predict([test_data])[0]*100))
print('MSE:',mse)
print('EVS:',evs)

Volatility: 61.67%
MSE: 0.008205501656895743
EVS: 0.8985428911410713


$$ {European Call Option Price Prediction} $$

In [ ]:
# simulate option prices
opt_price = []
for i in range(draws):
  p = european_option(S[i], K[i], T[i], r[i], d[i], sigma[i], opt_type[i])
  opt_price.append(p)
  if (i % 5000 == 0):
    print('Gnerated {} options'.format(i))
 
# create dataframe
options = pd.DataFrame({'S':S,
                        'K':K,
                        'T':T,
                        'r':r,
                        'd':d,
                        'sigma':sigma,
                        'type':opt_type,
                        'price':opt_price})
 
print(options.head())
print(options.shape)
options

Gnerated 0 options
           S          K     T         r         d     sigma  type      price
0  37.454012  35.955851  2.96  0.087205  0.016812  0.494521     0   5.354697
1  95.071431  57.993573  2.17  0.052723  0.047495  0.121695     0   0.007537
2  73.199394  81.251328  1.15  0.063254  0.005522  0.588094     1  15.783524
3  59.865848  77.226944  1.67  0.064447  0.094238  0.316078     0  19.196255
4  15.601864  21.374554  1.30  0.081980  0.007482  0.626968     0   6.233007
(1000, 8)


S           K     T         r         d     sigma  type      price
0    37.454012   35.955851  2.96  0.087205  0.016812  0.494521     0   5.354697
1    95.071431   57.993573  2.17  0.052723  0.047495  0.121695     0   0.007537
2    73.199394   81.251328  1.15  0.063254  0.005522  0.588094     1  15.783524
3    59.865848   77.226944  1.67  0.064447  0.094238  0.316078     0  19.196255
4    15.601864   21.374554  1.30  0.081980  0.007482  0.626968     0   6.233007
..         ...         ...   ...       ...       ...       ...   ...        ...
995   9.158207    9.799282  2.17  0.089189  0.036165  0.621106     1   2.568165
996  91.731358   77.971654  1.23  0.030605  0.012538  0.301800     1  19.476347
997  13.681863   16.144598  0.31  0.055188  0.025361  0.899743     1   1.868905
998  95.023735  119.729907  2.23  0.078941  0.064209  0.628407     1  20.665880
999  44.600577   63.332820  1.80  0.041622  0.027554  0.438649     1   4.851312

[1000 rows x 8 columns]

In [ ]:
X,y = options[options.columns.difference(['price'])], options['price']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
y_train = y_train.values.reshape(-1,1)
y_test = y_test.values.reshape(-1,1)

In [ ]:
scalerX = StandardScaler()
scalerX.fit(X_train)
X_train_scaled = scalerX.transform(X_train)
X_test_scaled = scalerX.transform(X_test)
scalerY = StandardScaler()
scalerY.fit(y_train)
y_train_scaled = scalerY.transform(y_train)
y_test_scaled = scalerY.transform(y_test)

reg = SVR()
reg.fit(X_train_scaled, y_train_scaled)

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


SVR(C=1.0, cache_size=200, coef0=0.0, degree=3, epsilon=0.1, gamma='scale',
    kernel='rbf', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

In [ ]:
y_train_hat_scaled = reg.predict(X_train_scaled)
y_train_hat = scalerY.inverse_transform(y_train_hat_scaled)
print('R_square:', r2_score(y_train, y_train_hat))
y_test_hat_scaled = reg.predict(X_test_scaled)
y_test_hat = scalerY.inverse_transform(y_test_hat_scaled)
print('R_square:', r2_score(y_test, y_test_hat))

R_square: 0.9480723411715527
R_square: 0.9244297418392151


In [ ]:
C_ep_ga_data = pd.DataFrame(columns = ('C', 'epsilon', 'gamma', 'training r2', 'test r2'))

training_r2score = []
test_r2score = []
C_settings = [1, 100]
epsilon_settings = [0.001, 0.01, 0.1]
gamma_settings = [0.01, 0.1]

for C in C_settings:
  for epsilon in epsilon_settings:
    for gamma in gamma_settings:
      # build the model
      reg = SVR(C=C, kernel = 'rbf', epsilon=epsilon, gamma=gamma)
      reg.fit(X_train_scaled, y_train_scaled)

      # r2 on the training set
      y_train_hat = scalerY.inverse_transform(reg.predict(X_train_scaled))
      training_r2score.append(r2_score(y_train, y_train_hat))

      # r2 on the test set
      y_test_hat = scalerY.inverse_transform(reg.predict(X_test_scaled))
      test_r2score.append(r2_score(y_test, y_test_hat))

      i = [C, epsilon, gamma, r2_score(y_train, y_train_hat), r2_score(y_test, y_test_hat)]

      C_ep_ga_data.loc[len(C_ep_ga_data)] = i
C_ep_ga_data

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C  epsilon  gamma  training r2   test r2
0     1.0    0.001   0.01     0.504558  0.508406
1     1.0    0.001   0.10     0.942616  0.930367
2     1.0    0.010   0.01     0.506435  0.510814
3     1.0    0.010   0.10     0.942975  0.930497
4     1.0    0.100   0.01     0.511139  0.516726
5     1.0    0.100   0.10     0.939038  0.924168
6   100.0    0.001   0.01     0.978060  0.970134
7   100.0    0.001   0.10     0.999620  0.991558
8   100.0    0.010   0.01     0.977992  0.969671
9   100.0    0.010   0.10     0.999610  0.990807
10  100.0    0.100   0.01     0.975588  0.968554
11  100.0    0.100   0.10     0.994643  0.979673

In [ ]:
C_ep_ga_data = pd.DataFrame(columns = ('C', 'epsilon', 'gamma', 'training rmse', 'test rmse'))

training_rmsescore = []
test_rmsescore = []
C_settings = [1, 100]
epsilon_settings = [0.001, 0.01, 0.1]
gamma_settings = [0.01, 0.1]

for C in C_settings:
  for epsilon in epsilon_settings:
    for gamma in gamma_settings:
      # build the model
      reg = SVR(C=C, kernel = 'rbf', epsilon=epsilon, gamma=gamma)
      reg.fit(X_train_scaled, y_train_scaled)

      # rmse on the training set
      y_train_hat = scalerY.inverse_transform(reg.predict(X_train_scaled))
      training_rmsescore.append(mean_squared_error(y_train, y_train_hat))

      # rmse on the test set
      y_test_hat = scalerY.inverse_transform(reg.predict(X_test_scaled))
      test_rmsescore.append(mean_squared_error(y_test, y_test_hat))

      i = [C, epsilon, gamma, mean_squared_error(y_train, y_train_hat), mean_squared_error(y_test, y_test_hat)]

      C_ep_ga_data.loc[len(C_ep_ga_data)] = i
C_ep_ga_data

/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py:760: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

C  epsilon  gamma  training rmse  test rmse
0     1.0    0.001   0.01      61.406951  60.594834
1     1.0    0.001   0.10       7.112370   8.583150
2     1.0    0.010   0.01      61.174257  60.297921
3     1.0    0.010   0.10       7.067830   8.567099
4     1.0    0.100   0.01      60.591177  59.569258
5     1.0    0.100   0.10       7.555865   9.347216
6   100.0    0.001   0.01       2.719328   3.681338
7   100.0    0.001   0.10       0.047147   1.040569
8   100.0    0.010   0.01       2.727804   3.738435
9   100.0    0.010   0.10       0.048301   1.133143
10  100.0    0.100   0.01       3.025714   3.876124
11  100.0    0.100   0.10       0.663963   2.505494

In [ ]:
num_training = int(0.8 * len(X))
X_train, y_train = X[:num_training], y[:num_training]
X_test, y_test = X[:num_training], y[:num_training]

sv_regressor = SVR(kernel='rbf', C=100.0, epsilon=0.001, gamma=0.10)

sv_regressor.fit(X_train, y_train)

y_pred = sv_regressor.predict(X_test)
X_test

K          S     T         d         r     sigma  type
0     35.955851  37.454012  2.96  0.016812  0.087205  0.494521     0
1     57.993573  95.071431  2.17  0.047495  0.052723  0.121695     0
2     81.251328  73.199394  1.15  0.005522  0.063254  0.588094     1
3     77.226944  59.865848  1.67  0.094238  0.064447  0.316078     0
4     21.374554  15.601864  1.30  0.007482  0.081980  0.626968     0
..          ...        ...   ...       ...       ...       ...   ...
795   65.409293  87.212391  0.99  0.032568  0.083137  0.050507     1
796  112.786312  93.211828  1.12  0.039414  0.084233  0.547305     0
797   54.817919  56.513318  2.59  0.060333  0.037324  0.890441     1
798   80.114845  69.665082  0.17  0.080168  0.063199  0.262721     1
799  109.777426  92.249938  1.62  0.046952  0.009537  0.472248     0

[800 rows x 7 columns]

In [ ]:
mse = mean_squared_error(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

test_data = [81.251328, 73.199394,	1.15,	0.005522,	0.063254,	0.588094,	1]
print('European Call Option Price:', sv_regressor.predict([test_data])[0])
print('MSE:',mse)
print('EVS:',evs)

European Call Option Price: 15.78469567423669
MSE: 0.5729972628116465
EVS: 0.9956655251693073


In [ ]:
mse = mean_squared_error(y_test, y_pred)
evs = explained_variance_score(y_test, y_pred)

test_data = [109.777426,	92.249938,	1.62,	0.046952,	0.009537,	0.472248,	0]
print('European Call Option Price:', sv_regressor.predict([test_data])[0])
print('MSE:',mse)
print('EVS:',evs)

European Call Option Price: 35.274502911283975
MSE: 0.5729972628116465
EVS: 0.9956655251693073
